<a href="https://colab.research.google.com/github/stakunlena/ich_final_project/blob/main/03_product_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Продуктовый анализ

## Импорт библиотек и исходных данных

In [1]:
# Импортируем основные библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)
sns.set(style="whitegrid", palette="Blues_d") # Устанавливаем единый стиль для всех диаграмм

# Загружаем исходные данные
import os
from google.colab import drive # Импортируем библиотеку для работы с Google Drive

# Подключаем Google Drive
drive.mount('/content/drive')

# Путь к папке с данными
base_path = '/content/drive/MyDrive/P. Project 07.11/csv/'
postfix = '20251103_0053' # чтобы задавать имена версий исходных файлов

# Загружаем данные
#df_contacts = pd.read_csv(base_path + 'df_contacts_clean_'+ postfix +'.csv', sep=";", encoding="utf-8-sig", parse_dates=["created_time", "modified_time"])
#df_calls = pd.read_csv(base_path + 'df_calls_clean_'+ postfix +'.csv', sep=";", encoding="utf-8-sig", parse_dates=["call_start_time"])
df_deals = pd.read_csv(base_path + 'df_deals_clean_'+ postfix +'.csv', sep=";", encoding="utf-8-sig", parse_dates=["created_time", "closing_date"])
df_spend = pd.read_csv(base_path + 'df_spend_clean_'+ postfix +'.csv', sep=";", encoding="utf-8-sig", parse_dates=["date"])

# Проверяем загрузку
#print(f"Contacts: {df_contacts.shape} | Calls: {df_calls.shape} | Deals: {df_deals.shape} | Spend: {df_spend.shape}")
print(f"Deals: {df_deals.shape} | Spend: {df_spend.shape}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Deals: (21593, 25) | Spend: (19862, 12)


## Юнит-экономика по продуктам

In [57]:
# 0. Целевые продукты
TARGET = {"digital marketing", "ux/ui design", "web developer"}

# подготовка
df_deals["initial_amount_paid"] = pd.to_numeric(df_deals["initial_amount_paid"], errors="coerce").fillna(0)
df_deals["offer_total_amount"] = pd.to_numeric(df_deals["offer_total_amount"], errors="coerce").fillna(0)
df_deals["payment_done"] = df_deals["initial_amount_paid"] > 0
df_deals["stage_normalized"] = df_deals["stage_normalized"].str.lower().str.strip()

# фильтр только целевых продуктов
deals_t = df_deals[df_deals["product"].isin(TARGET)].copy()

# 1. Подсчёт сделок (T) и клиентов (B)
# Проверяем, есть ли поле идентификатора клиента
client_col = "contact_name" if "contact_name" in df_deals.columns else "contactid"

b_t_rev_aov = (
    deals_t.groupby("product", as_index=False)
    .agg(
        t=("id", "count"),  # общее число сделок по продукту
        b=(client_col, "nunique"),  # уникальные клиенты
        revenue=("initial_amount_paid", "sum"),
        aov=("initial_amount_paid", lambda x: round(x[x>0].mean(), 2))
    )
)
b_t_rev_aov["revenue"] = b_t_rev_aov["revenue"].round(2)
b_t_rev_aov["apc"] = np.where(b_t_rev_aov["b"]>0, b_t_rev_aov["t"]/b_t_rev_aov["b"], 0).round(2)

# 2. CAC и AC
spend_by_camp = df_spend.groupby("campaign", as_index=False).agg(spend=("spend", "sum"))
leads_by_camp = df_deals.groupby("campaign", as_index=False).agg(leads=("id", "count"))
cac = spend_by_camp.merge(leads_by_camp, on="campaign", how="outer")
cac["spend"] = cac["spend"].fillna(0)
cac["leads"] = cac["leads"].fillna(0)
cac["cac_per_lead"] = np.where(cac["leads"]>0, cac["spend"]/cac["leads"], 0).round(2)

prod_camp = (
    deals_t.groupby(["product", "campaign"], as_index=False)
    .agg(leads=("id", "count"))
)
prod_camp = prod_camp.merge(cac[["campaign", "cac_per_lead"]], on="campaign", how="left")
prod_camp["cac_per_lead"] = prod_camp["cac_per_lead"].fillna(0)
prod_camp["ac_attributed"] = prod_camp["leads"] * prod_camp["cac_per_lead"]

ac_by_product = (
    prod_camp.groupby("product", as_index=False)
    .agg(ac_total=("ac_attributed", "sum"))
)
ac_by_product["ac_total"] = ac_by_product["ac_total"].round(2)

# 3. UA по стадии сделки
ua_by_product = []
for p in TARGET:
    mask = (
        (df_deals["product"].isna()) # без продукта
        | (df_deals["product"] == p) # сам продукт
        #| (df_deals["payment_done"] == False) # нет оплаты
        #| (df_deals["offer_total_amount"] == 0) # нет полной стоимости предложения
        #| (df_deals["stage_normalized"] != "active student") # не активный студент
    )
    ua = df_deals.loc[mask].shape[0]
    ua_by_product.append({"product": p, "ua": ua})
ua_by_product = pd.DataFrame(ua_by_product)

# 4. Объединяем все данные
prod = (
    ua_by_product
    .merge(b_t_rev_aov, on="product", how="left")
    .merge(ac_by_product, on="product", how="left")
)

# 5. Расчёт метрик юнит-экономики
prod["ac"] = prod["ac_total"].fillna(0)
prod["c1"] = np.round(np.where(prod["ua"]>0, prod["b"]/prod["ua"], 0.0), 4)
prod["cogs_per_client"] = 0.0
prod["cltv"] = np.round((prod["aov"].fillna(0) - prod["cogs_per_client"]) * prod["apc"], 2)
prod["ltv"] = np.round(prod["cltv"] * prod["c1"], 2)
prod["cpa"] = np.round(np.where(prod["ua"]>0, prod["ac_total"]/prod["ua"], 0.0), 2)
prod["cm"] = np.round((prod["cltv"] * prod["c1"] - prod["cpa"]) * prod["ua"], 2)

# 6. Финальная таблица
cols = ["product","ua","b","t","apc","c1","ac","cpa","revenue","aov","cltv","ltv","cm"]
prod = prod[cols].sort_values("revenue", ascending=False)

# 7. Форматирование вывода
int_cols = prod.select_dtypes(include=["int64"]).columns
float_cols = prod.select_dtypes(include=["float64"]).columns
format_dict = {}
format_dict.update({col: "{:,.0f}" for col in int_cols})
format_dict.update({col: "{:,.2f}" for col in float_cols})

styled = (
    prod.style
    .format(format_dict)
    .background_gradient(subset=["revenue"], cmap="Greens")
    .background_gradient(subset=["cm"], cmap="Blues")
    .set_caption("Юнит-экономика по продуктам (APC — среднее число сделок на одного клиента в когорте)")
    .hide(axis="index")
)

display(styled)


product,ua,b,t,apc,c1,ac,cpa,revenue,aov,cltv,ltv,cm
digital marketing,"19,991","1,712","1,990",1.16,0.09,"10,445.69",0.52,"2,145,450.00","1,179.47","1,368.19",117.12,"2,330,891.91"
ux/ui design,"19,023",942,"1,022",1.08,0.05,"4,812.33",0.25,"1,150,200.00","1,238.11","1,337.16",66.19,"1,254,365.59"
web developer,"18,576",540,575,1.06,0.03,"3,482.11",0.19,"567,450.00","1,074.72","1,139.20",33.15,"612,278.33"
